In [1]:
from moviepy.editor import VideoFileClip

video = VideoFileClip('D:/Projects/external_data/Previous/20221020_121933.mp4')
video.write_images_sequence('frame%05d.png',fps = 10, logger='bar')

Moviepy - Writing frames frame%05d.png.


Moviepy - Done writing frames frame%05d.png.


['frame00000.png',
 'frame00001.png',
 'frame00002.png',
 'frame00003.png',
 'frame00004.png',
 'frame00005.png',
 'frame00006.png',
 'frame00007.png',
 'frame00008.png',
 'frame00009.png',
 'frame00010.png',
 'frame00011.png',
 'frame00012.png',
 'frame00013.png',
 'frame00014.png',
 'frame00015.png',
 'frame00016.png',
 'frame00017.png',
 'frame00018.png',
 'frame00019.png',
 'frame00020.png',
 'frame00021.png',
 'frame00022.png']

In [1]:
import numpy as np
import os
import imutils
import dlib
import cv2 

import imageio
from imutils import face_utils

In [2]:
import os
file_list = os.listdir('D:/Projects/external_data/previous')

In [3]:
def crop_and_save_image(img, img_path, write_img_path, img_name):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('D:/Projects/shape_predictor_68_face_landmarks/shape_predictor_68_face_landmarks/shape_predictor_68_face_landmarks.dat')
    # load the input image, resize it, and convert it to grayscale

    image = cv2.imread(img_path)
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    if len(rects) > 1:
        print("Error")
        return
    if len(rects) < 1:
        print( "ERROR: no faces detected")
        return
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        name, i, j = 'mouth', 48, 68
        # clone = gray.copy()
        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))        
        roi = gray[y:y+h, x:x+w]
        roi = imutils.resize(roi, width = 250, inter=cv2.INTER_CUBIC)        
        print( write_img_path)
        cv2.imwrite(write_img_path, roi)
        

In [4]:
directory = 'D:/Projects/external_data/previous/frames/'
dir_temp = 'D:/Projects/external_data/previous/cropped/'
for img_name in file_list:
    image = imageio.imread(directory + '' + img_name)
    crop_and_save_image(image, directory + '' + img_name,dir_temp + '' + img_name, img_name)

C:\Users\deepdesk\AppData\Local\Temp\ipykernel_6644\527829137.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(directory + '' + img_name)


D:/Projects/external_data/previous/cropped/frame00006.png
D:/Projects/external_data/previous/cropped/frame00007.png
D:/Projects/external_data/previous/cropped/frame00008.png
D:/Projects/external_data/previous/cropped/frame00009.png
D:/Projects/external_data/previous/cropped/frame00010.png
D:/Projects/external_data/previous/cropped/frame00011.png
D:/Projects/external_data/previous/cropped/frame00012.png
D:/Projects/external_data/previous/cropped/frame00013.png
D:/Projects/external_data/previous/cropped/frame00014.png
D:/Projects/external_data/previous/cropped/frame00015.png
D:/Projects/external_data/previous/cropped/frame00016.png
D:/Projects/external_data/previous/cropped/frame00017.png
D:/Projects/external_data/previous/cropped/frame00018.png
D:/Projects/external_data/previous/cropped/frame00019.png
D:/Projects/external_data/previous/cropped/frame00020.png


Loading The Saved Model

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
savedModel=load_model('D:/Projects/MIRACL-VC1/models/cnn_adagrad_e45_bc1/model.h5')

PreProcessing

In [6]:
max_seq_length = 22
MAX_WIDTH = 100
MAX_HEIGHT = 100

In [7]:
from skimage.transform import resize
import time
sequence = []
for img_name in file_list:        
    image = imageio.imread(dir_temp + '/' + img_name)
    image = resize(image, (MAX_WIDTH, MAX_HEIGHT))
    image = 255 * image
    # Convert to integer data type pixels.
    image = image.astype(np.uint8)
    sequence.append(image)                        
pad_array = [np.zeros((MAX_WIDTH, MAX_HEIGHT))]                            
sequence.extend(pad_array * (max_seq_length - len(sequence)))
sequence = np.array(sequence)

C:\Users\deepdesk\AppData\Local\Temp\ipykernel_6644\3515270067.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(dir_temp + '/' + img_name)


In [8]:
def normalize_it(X):
    v_min = X.min(axis=(2, 3), keepdims=True)
    v_max = X.max(axis=(2, 3), keepdims=True)
    X = (X - v_min)/(v_max - v_min)
    X = np.nan_to_num(X)
    return X

In [9]:
X_test = []
X_test.append(sequence)
X_test = np.array(X_test)
X_test = normalize_it(X_test)
X_test = np.expand_dims(X_test, axis=4)
ypred = savedModel.predict(X_test)
words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']  
predicted_words = [words[i] for i in np.argmax(ypred, axis=1)]
print(predicted_words)

C:\Users\deepdesk\AppData\Local\Temp\ipykernel_6644\3382126703.py:4: RuntimeWarning: invalid value encountered in true_divide
  X = (X - v_min)/(v_max - v_min)


1/1 [==============================] - 3s 3s/step
['Stop']


In [10]:
ypred

array([[1.3020193e-03, 1.1207085e-03, 2.7790403e-03, 1.2108639e-04,
        4.1729319e-01, 1.1743272e-04, 1.3977316e-03, 5.7120907e-01,
        1.2432352e-05, 4.6472815e-03]], dtype=float32)

In [11]:
# release GPU memory
from keras import backend as K
K.clear_session()

from numba import cuda
cuda.select_device(0)
cuda.close()

In [ ]:
# # import the opencv library
# import cv2
  
  
# # define a video capture object
# vid = cv2.VideoCapture(0)
  
# while(True):
      
#     # Capture the video frame
#     # by frame
#     ret, frame = vid.read()
  
#     # Display the resulting frame
#     cv2.imshow('frame', frame)
      
#     # the 'q' button is set as the
#     # quitting button you may use any
#     # desired button of your choice
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
  
# # After the loop release the cap object
# vid.release()
# # Destroy all the windows
# cv2.destroyAllWindows()